# EDA

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_df = pd.read_csv("data/hate/hate-text.csv")
data_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3901 entries, 0 to 3900
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   case_id       3901 non-null   int64 
 1   test_case     3901 non-null   object
 2   label_gold    3901 non-null   object
 3   target_ident  3606 non-null   object
dtypes: int64(1), object(3)
memory usage: 122.0+ KB


In [21]:
# We note that there are rows that are identified as "non-hateful" but not against a certain group, we fill those rows with "no group"
data_df["target_ident"].fillna(value="no group", inplace=True)

In [22]:
data_df["label_gold"].value_counts()

label_gold
hateful        2659
non-hateful    1242
Name: count, dtype: int64

In [23]:
data_df["target_ident"].value_counts()

target_ident
gay people         577
women              535
disabled people    510
Muslims            510
black people       504
trans people       485
immigrants         485
no group           295
Name: count, dtype: int64

In [24]:
X, y = data_df["test_case"], data_df[["label_gold", "target_ident"]]
X.shape, y.shape

((3901,), (3901, 2))